In [1]:
import pandas as pd
import os
import sys

sys.path.append('/Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils
import numpy as np

pathPrefix = '/Users/friedman/Desktop/mnt'
from collections import Counter

In [2]:
def compare_presence_absense_of_mutations_between_pairs(maf, firstSample, secondSample):
    listOfDicts = []
    firstSampleMaf = maf[maf['Tumor_Sample_Barcode'] == firstSample]
    secondSampleMuts = set(maf[maf['Tumor_Sample_Barcode'] == secondSample]['varUuid'])
    method = firstSampleMaf['clonalityMethod'].iloc[0]
    
    ccfFracDict = {}
    if method == 'FACETS_CCF':
        ccfFracDict = dict(zip(firstSampleMaf['varUuid'], firstSampleMaf['ccf_Mcopies_lower']))
    else:
        ccfFracDict = dict(zip(firstSampleMaf['varUuid'], firstSampleMaf['t_var_freq']))
    for varUuid in set(firstSampleMaf['varUuid']):
        mutationLost = True
        if varUuid in secondSampleMuts:
            mutationLost = False
        listOfDicts.append({'ccf': ccfFracDict[varUuid], 'varUuid': varUuid, 'isLost': mutationLost,
        'comparisson': firstSample + '->' + secondSample, 'clonalityMethod': method})
        
    return pd.DataFrame(listOfDicts)
        
        

In [4]:
#SIGNATURE DIFFERENCES
multipleSampleSignatures = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/HypermutantMultipleSamplesBranchMutationalSignatures.tsv')
multipleSampleSignatures['Nmut'] = multipleSampleSignatures['Number of Mutations']
multipleSampleSignatures = mutationSigUtils.merge_signature_columns(multipleSampleSignatures, mode='Stratton', drop=True, mmrAgingMerge=True, smokingMerge=False, confidence=False, mean=True, prefix='Signature.')
multipleSampleSignatures['dominantSignature'] = multipleSampleSignatures.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='Signature.', notEnoughMuts= True), axis=1)

In [6]:
mafWithClonalityInfoRed = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/clonallyMarkedMutsHypermutators.tsv')
mafWithClonalityInfoRed['Tumor_Sample_Barcode'] = mafWithClonalityInfoRed['Tumor_Sample_Barcode_bak']

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (17,18,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
pairsInfo = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/pairsDataFromCraig.txt')

In [34]:
reload(get_gene_and_cohort_list_utils)
pairsSamples = get_gene_and_cohort_list_utils.get_hypermutator_pairs_samples()

In [51]:
mafWithClonalityInfoRed['pid'] = mafWithClonalityInfoRed['Tumor_Sample_Barcode_bak'].apply(lambda x: x[:9])
mafWithClonalityInfoMultiples = mafWithClonalityInfoRed[mafWithClonalityInfoRed['Tumor_Sample_Barcode_bak'].isin(pairsSamples)]

In [52]:
mafWithClonalityInfoMultiples['varUuid'] = mafWithClonalityInfoMultiples.apply(lambda row: str(row['Chromosome'] + '_'
                + str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2'])), axis=1)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
df = compare_presence_absense_of_mutations_between_pairs(mafWithClonalityInfoRed,'P-0004379-T02-IM6', 'P-0004379-T01-IM5')


In [61]:
df.to_csv('/Users/friedman/Desktop/WORK/bespokePairComparisson.tsv', index=False, sep='\t')

In [65]:
#pairsInfo['patient']
pairsInfo[pairsInfo['patient'] == 'P-0008682']

,tumor_sample,patient,tumor_IM_ver,tumor_bam_coverage,normal_sample,normal_bam_coverage,tumor_bam_path,normal_bam_path,PartA,Cancer.Type,...,PRIMARY_SITE,TUMOR_TYPE_NAME,METASTASIS_SITE,Low Tumor Ind,Fail Ind,proc_date,rep_date,darwin_prim_met,CuratedSampleType,dmp_calls_exist


In [57]:
compDf 

,ccf,clonalityMethod,comparisson,isLost,varUuid
0,0.340267,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,4_187628637
1,0.319461,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,False,11_119149251
2,0.330998,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,18_56412904
3,0.301781,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,15_42005410
4,0.337093,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,19_7142931
5,0.310440,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,False,19_50906366
6,0.288100,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,17_29560198
7,0.328421,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,X_53241016
8,0.350602,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,True,3_52584803
9,0.324910,VAF_CLUSTERING,P-0004379-T01-IM5->P-0004379-T02-IM6,False,5_112176828


In [28]:
#mafWithClonalityInfoRed[mafWithClonalityInfoRed['tcn'] != 2][['isBalanced', 't_var_freq', 'FlatGenome', 'isClonal', 'tcn', 'lcn', 'ccf_Mcopies_upper']]
#print mafWithClonalityInfoRed.columns.values

for case in set(mafWithClonalityInfoRed['Tumor_Sample_Barcode_bak']):
    caseMaf = mafWithClonalityInfoRed[mafWithClonalityInfoRed['Tumor_Sample_Barcode_bak'] == case]
    method = caseMaf['clonalityMethod'].iloc[0]
    
    print case, method 
    print Counter(caseMaf['isClonal'])
    if method == 'FACETS_CCF':
        print max(caseMaf[caseMaf['isClonal'] == False]['ccf_Mcopies_lower'])
    else:
        print max(caseMaf[caseMaf['isClonal'] == False]['t_var_freq'])
    print '___________________'
    
    #print np.nanmean(caseMaf[caseMaf['ccf_Mcopies_lower'].notnull()]['ccf_Mcopies_lower']), caseMaf[caseMaf['ccf_Mcopies_lower'].isnull()].shape[0]



P-0025195-T01-IM6 FACETS_CCF
Counter({False: 35, True: 1})
0.53
___________________
P-0023479-T01-IM6 FACETS_CCF
Counter({False: 24, True: 8})
0.779
___________________
P-0037006-T01-IM6 FACETS_CCF
Counter({True: 26, False: 10})
0.76
___________________
P-0002957-T01-IM3 FACETS_CCF
Counter({False: 11, True: 9})
0.792
___________________
P-0001494-T01-IM3 FACETS_CCF
Counter({True: 32, False: 25})
0.787
___________________
P-0028485-T01-IM6 FACETS_CCF
Counter({True: 65, False: 50})
0.775
___________________
P-0000012-T03-IM3 FACETS_CCF
Counter({False: 26, True: 4})
0.776
___________________
P-0034671-T02-IM6 FACETS_CCF
Counter({True: 39, False: 21})
0.794
___________________
P-0012804-T01-IM5 FACETS_CCF
Counter({True: 39, False: 10})
0.752
___________________
P-0000448-T01-IM3 FACETS_CCF
Counter({True: 58, False: 17})
0.772
___________________
P-0003610-T01-IM5 FACETS_CCF
Counter({True: 13, False: 12})
0.751
___________________
P-0010671-T01-IM5 VAF_CLUSTERING
Counter({False: 16, True: 1

ValueError: max() arg is an empty sequence